 # Case Study: Virtual Assistant for IlluminatiMart

This Jupyter Notebook demonstrates the implementation of the Illuminati Assistant, a virtual assistant designed for IlluminatiMart, an AI-driven online grocery platform. The purpose of this chatbot is to enhance the customer shopping experience by offering personalized assistance. It helps users by answering queries, guiding them through product searches, and providing relevant product recommendations based on customer preferences and previous shopping behavior. This notebook outlines the chatbot's development process, including the integration of natural language processing (NLP) techniques to ensure accurate, context-aware interactions.

In [2]:
import os
from openai import OpenAI
import gradio as gr

client = OpenAI(api_key=os.environ["OPENAI"])

# Initialize the chat history with the system message
initial_history = [
    {
        "role": "system", 
        "content": "You are a helpful virtual assistant - Illuminati Assistant - for the online grocery store IlluminatiMart. You can answer questions regarding IlluminatiMart's online groceries, products, checkout, purchase, shipment, return policies, and so on. Any question that is not related to this topic, answer politely that you have no knowledge about the topic. If you make a mistake, you apologize that you are only a virtual assistant and you can give the contact details of the customer support."
    }
]

def openai_chatbot(message, history):
    # If a user message is passed, append it to the history
    if message:
        history.append({"role": "user", "content": message})
    
    # Create the response using OpenAI's GPT-4
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=history
    )
    
    # Extract the assistant's message from the response
    assistant_message = response.choices[0].message.content
    
    # Append the assistant's message to the history
    history.append({"role": "assistant", "content": assistant_message})
    
    return assistant_message, history

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Olivia 🍃 - Your personal AI-Assistant by IlluminatiMart")
    chatbot = gr.Chatbot(type="messages")  # Set type to 'messages'
    msg = gr.Textbox(placeholder="Type your message to Olivia here...")
    
    gr.Markdown(
        """
        **Disclaimer:** This is an AI-powered assistant by IlluminatiMart. I can make mistakes, please double-check any information before purchase decisions. Please do not share personal or sensitive information. 
        For urgent matters, contact customer support directly.
        """
    )
    
    clear = gr.Button("Clear")

    # Automatically generate the introduction message when the chat is loaded
    def init_chat():
        assistant_intro = "Hello and welcome! I'm Olivia 🍃, your personal AI-powered virtual assistant from IlluminatiMart. I'm here to assist you with anything related to your shopping experience 🛒. I can help you: 1️⃣ finding products, 2️⃣ check orders, or 3️⃣ answer your questions. My goal is to make your online grocery shopping more convenient and personalized 🤩. How can I assist you today?"
        history = initial_history.copy()  # Initialize history with the system message
        history.append({"role": "assistant", "content": assistant_intro})  # Assistant introduces itself
        return history, [{"role": "assistant", "content": assistant_intro}]  # Return the updated history and the initial message for the chatbot

    def respond(message, chat_history):
        response, updated_history = openai_chatbot(message, chat_history)
        return "", updated_history, updated_history

    # Initialize the chat with the assistant's introduction
    history_state = gr.State(value=[])  # Create a state for the chat history

    # Hook up the user input and response to the chat interface
    msg.submit(respond, [msg, history_state], [msg, chatbot, history_state])
    
    # Clear button resets the chatbot and history state
    clear.click(lambda: ([], []), None, [chatbot, history_state], queue=False)

    # Initialize the chatbot with the intro message
    demo.load(init_chat, inputs=None, outputs=[history_state, chatbot])

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
